### Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('../')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

### Configuration

In [3]:
from sv_system.utils.parser import set_train_config
import easydict

args = easydict.EasyDict(dict(dataset="voxc1_fbank_xvector", 
                              data_folder="/dataset/SV_sets/voxceleb12/feats/fbank64_vad/",
                              input_frames=400, splice_frames=[200, 400], stride_frames=1, 
                              input_format='fbank', input_dim=65, random_clip=True,
                              n_epochs=200, lrs=[0.1, 0.01], lr_schedule=[20], seed=1337,
                              no_eer=False, batch_size=128,
                              gpu_no=[0], cuda=True, num_workers=4,
                              arch="tdnn_conv", loss="softmax",
                             ))
config = set_train_config(args)

### Dataset

In [20]:
from sv_system.data.feat_dataset import FeatDataset
# dev_df = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_dev.csv")
dev_df = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_si.csv")
dev_df = dev_df.sample(frac=1.0)
dev_train_df = dev_df[dev_df.set == 'train']
dev_val_df = dev_df[dev_df.set == 'val']
eval_df = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_eval.csv") 

In [21]:
dev_train_dataset = FeatDataset.read_df(config, dev_train_df, 'train')
dev_val_dataset = FeatDataset.read_df(config, dev_val_df, 'test')
eval_dataset = FeatDataset.read_df(config, eval_df, 'test')

### Dataloader

In [22]:
from sv_system.data.dataloader import init_default_loader 
dev_train_dataloader = init_default_loader(config, dev_train_dataset, shuffle=True, var_len=False) 
dev_val_dataloader = init_default_loader(config, dev_val_dataset, shuffle=False, var_len=False) 
eval_dataloader = init_default_loader(config, eval_dataset, shuffle=False, var_len=True) 

### Model

In [23]:
from tdnn_models import tdnn_xvector

### Train Model

In [24]:
import torch

model = tdnn_xvector(config, 512, n_labels=len(dev_df.label.unique()))
# saved_model = torch.load("trained_models/vox1_dev_tdnn_xvector_held_out.pt")
# model.load_state_dict(saved_model)

if not config['no_cuda']:
    model = model.cuda()

In [25]:
from tensorboardX import SummaryWriter
writer = SummaryWriter("logs/self_paced_hard_voxc1_si")
model_path = "trained_models/voxc1_si_tdnn_xvector_spl_hard.pt"

In [27]:
import torch.nn as nn
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR
from sklearn.metrics import roc_curve
import torch.nn.functional as F

optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
criterion = nn.CrossEntropyLoss(reduce=False)
plateau_scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5)
step_scheduler = MultiStepLR(optimizer, [30], 0.1)

k = 6

for epoch_idx in range(0, config['n_epochs']):
    print("-"*30)
    curr_lr = optimizer.state_dict()['param_groups'][0]['lr']
    print("curr_lr: {}".format(curr_lr))
    
    n_sel_samples = 0
# =============== train code #===============
    model.train()
    loss_sum = 0
    n_corrects = 0
    total = 0
    for batch_idx, (X, y) in enumerate(dev_train_dataloader):
        if not config['no_cuda']:
            X = X.cuda()
            y = y.cuda()

        optimizer.zero_grad()
        logit = model(X)
        loss = criterion(logit, y)
        ### spl masking
        spl_mask = loss.gt(k)
        masked_loss = torch.masked_select(loss, spl_mask).mean()
        n_sel_samples += spl_mask.sum().item()
        masked_loss.backward()
        optimizer.step()
                        
        loss_sum += masked_loss.item()
        n_corrects += logit.max(1)[1].eq(y).sum().item()
        total += y.size(0)
        if (batch_idx+1) % 100 == 0:
            print("Batch {}/{}\t Loss {:.6f}" \
                  .format(batch_idx+1, len(dev_train_dataloader), loss_sum / total))
    train_loss = loss_sum / total
    train_acc = n_corrects / total
    plateau_scheduler.step(train_loss)
    
    print("epoch #{}, train loss: {:.4f}, train acc: {:.4f}".format(epoch_idx, train_loss, train_acc))
    writer.add_scalar("train/loss", train_loss, epoch_idx+1)
    writer.add_scalar("train/acc", train_acc, epoch_idx+1)
    
    k = k/1.2
    print("sel_samples/total: {}/{}".format(n_sel_samples, total))

#=============== dev_val code #===============
    model.eval()
    loss_sum = 0
    n_corrects = 0
    total = 0
    for batch_idx, (X, y) in enumerate(dev_val_dataloader):
        if not config['no_cuda']:
            X = X.cuda()
            y = y.cuda()

        logit = model(X)
        loss = criterion(logit, y)
        loss_sum += loss.mean().item()
        n_corrects += logit.max(1)[1].eq(y).sum().item()
        total += y.size(0)
    val_loss = loss_sum / total
    val_acc = n_corrects / total
    
    
    print("epoch #{}, val loss: {:.4f}, val acc: {:.4f}".format(epoch_idx, val_loss, val_acc))
    writer.add_scalar("val/loss", val_loss, epoch_idx+1)
    writer.add_scalar("val/acc", val_acc, epoch_idx+1)
    torch.save(model.state_dict(), model_path)

------------------------------
curr_lr: 0.01


/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Batch 100/1047	 Loss 0.053281
Batch 200/1047	 Loss 0.052417
Batch 300/1047	 Loss 0.051760
Batch 400/1047	 Loss 0.051301
Batch 500/1047	 Loss 0.050988
Batch 600/1047	 Loss 0.050781
Batch 700/1047	 Loss 0.050645
Batch 800/1047	 Loss 0.050554
Batch 900/1047	 Loss 0.050488
Batch 1000/1047	 Loss 0.050452
epoch #0, train loss: 0.0504, train acc: 0.0529
sel_samples/total: 41281/134000
epoch #0, val loss: 0.0447, val acc: 0.0201
------------------------------
curr_lr: 0.01
Batch 100/1047	 Loss 0.043362
Batch 200/1047	 Loss 0.043235
Batch 300/1047	 Loss 0.043266
Batch 400/1047	 Loss 0.043316
Batch 500/1047	 Loss 0.043348
Batch 600/1047	 Loss 0.043434
Batch 700/1047	 Loss 0.043458
Batch 800/1047	 Loss 0.043477
Batch 900/1047	 Loss 0.043477
Batch 1000/1047	 Loss 0.043510
epoch #1, train loss: 0.0435, train acc: 0.1708
sel_samples/total: 29851/134000
epoch #1, val loss: 0.0381, val acc: 0.0750
------------------------------
curr_lr: 0.01
Batch 100/1047	 Loss 0.037675
Batch 200/1047	 Loss 0.037688


In [127]:
# See, Fr features
fr_feats = []
model.eval()
total = 0
for batch_idx, (X, y) in enumerate(dev_val_dataloader):
    if not config['no_cuda']:
        X = X.cuda()
        y = y.cuda()

    fr_feat = model.fr_feat(X).cpu().detach()
    fr_feats.append(fr_feat)
    break

###  Freeze Model & FineTurne

In [117]:
freezed_params = []
for param_name, param in model.named_parameters():
    if param_name not in ['classifier.2.weight', 'classifier.2.bias']:
        freezed_params.append(param)
        
for param in freezed_params:
    param.requires_grad = False

In [121]:
# =============== fine_tune code #===============
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
for epoch_idx in range(0, 10):
    print("-"*30)
    curr_lr = optimizer.state_dict()['param_groups'][0]['lr']
    print("curr_lr: {}".format(curr_lr))
    model.train()
    loss_sum = 0
    n_corrects = 0
    total = 0
    for batch_idx, (X, y) in enumerate(held_out_train_dataloader):
        if not config['no_cuda']:
            X = X.cuda()
            y = y.cuda()

        optimizer.zero_grad()
        logit = model(X)
        loss = criterion(logit, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        n_corrects += logit.max(1)[1].eq(y).sum().item()
        total += y.size(0)
    #         if (batch_idx+1) % 1000 == 0:
    #             print("Batch {}/{}\t Loss {:.6f}" \
    #                   .format(batch_idx+1, len(si_loader), loss_sum / total))
    train_loss = loss_sum / total
    train_acc = n_corrects / total

    print("epoch #{}, train loss: {:.4f}, train acc: {:.4f}".format(epoch_idx, train_loss, train_acc))

------------------------------
curr_lr: 0.01
epoch #0, train loss: 0.0219, train acc: 0.9818
------------------------------
curr_lr: 0.01
epoch #1, train loss: 0.0217, train acc: 0.9818
------------------------------
curr_lr: 0.01
epoch #2, train loss: 0.0214, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #3, train loss: 0.0210, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #4, train loss: 0.0206, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #5, train loss: 0.0202, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #6, train loss: 0.0198, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #7, train loss: 0.0194, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #8, train loss: 0.0190, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #9, train loss: 0.0186, train acc: 0.9848


In [1]:
#=============== dev_val code #===============
model.eval()
loss_sum = 0
n_corrects = 0
total = 0
for batch_idx, (X, y) in enumerate(dev_val_dataloader):
    if not config['no_cuda']:
        X = X.cuda()
        y = y.cuda()

    logit = model(X)
    loss = criterion(logit, y)
    loss_sum += loss.item()
    n_corrects += logit.max(1)[1].eq(y).sum().item()
    total += y.size(0)
val_loss = loss_sum / total
val_acc = n_corrects / total
print("epoch #{}, val loss: {:.4f}, val acc: {:.4f}".format(epoch_idx, val_loss, val_acc))

NameError: name 'model' is not defined

### SV Test

### LDA on embedding

In [20]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

global_mean = si_embeds.mean(0)
clf = LDA(solver='svd', n_components=200)
clf.fit(si_embeds - global_mean, si_key_df.label)

si_embeds = clf.transform(si_embeds - global_mean).astype(np.float32)

sv_embeds = clf.transform(sv_embeds - global_mean).astype(np.float32)

si_dataset, embed_dim, n_labels = embedToDataset(si_embeds.reshape(-1,200), si_key_df)
sv_dataset, _, _ = embedToDataset(sv_embeds, sv_key_df)